In [ ]:
import os
import copy

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
from tqdm.auto import tqdm
from scipy.stats import linregress

In [ ]:
f = FAIR()

In [ ]:
f.define_time(1850, 2050, 1)

In [ ]:
scenarios = ['esm-bell-750PgC', 'esm-bell-1000PgC', 'esm-bell-2000PgC']

In [ ]:
# do one at a time
#scenarios = [scenarios[0]]
f.define_scenarios(scenarios)
fair_params_df = pd.read_csv('../data/fair2.1-parameters/calibration-1.4.0/calibrated_constrained_parameters.csv', index_col=0)

In [ ]:
f.define_configs(list(fair_params_df.index))

In [ ]:
species = ['CO2', 'CH4', 'N2O']
properties = {
    "CO2": {
        'type': 'co2',
        'input_mode': 'emissions',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    },
    "CH4": {
        'type': 'ch4',
        'input_mode': 'emissions',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    },
    "N2O": {
        'type': 'n2o',
        'input_mode': 'emissions',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    }
}

In [ ]:
# declare species and properties
f.define_species(species, properties)

In [ ]:
f.allocate()

In [ ]:
f.fill_from_rcmip()

In [ ]:
f.concentration.loc[dict(specie='CH4')] = 808.2490285
f.concentration.loc[dict(specie='N2O')] = 273.021047

In [ ]:
# Get default species configs
f.fill_species_configs()

# Climate response
fill(f.climate_configs['ocean_heat_capacity'], fair_params_df.loc[:,'clim_c1':'clim_c3'])
fill(f.climate_configs['ocean_heat_transfer'], fair_params_df.loc[:,'clim_kappa1':'clim_kappa3'])
fill(f.climate_configs['deep_ocean_efficacy'], fair_params_df.loc[:,'clim_epsilon'])
fill(f.climate_configs['gamma_autocorrelation'], fair_params_df.loc[:,'clim_gamma'])
fill(f.climate_configs['stochastic_run'], False)

# carbon cycle
fill(f.species_configs['iirf_0'], fair_params_df.loc[:, 'cc_r0'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_airborne'], fair_params_df.loc[:, 'cc_rA'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_uptake'], fair_params_df.loc[:, 'cc_rU'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_temperature'], fair_params_df.loc[:, 'cc_rT'].values.squeeze(), specie='CO2')

# Scale CO2 forcing based on its 4xCO2 calibration
fill(f.species_configs["forcing_scale"], fair_params_df["fscale_CO2"].values.squeeze(), specie='CO2')

# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(f.species_configs['baseline_concentration'], 284.3169988, specie='CO2')
fill(f.species_configs['baseline_concentration'], 808.2490285, specie='CH4')
fill(f.species_configs['baseline_concentration'], 273.021047, specie='N2O')

In [ ]:
fill(f.species_configs['forcing_reference_concentration'], 284.3169988, specie='CO2')
fill(f.species_configs['forcing_reference_concentration'], 808.2490285, specie='CH4')
fill(f.species_configs['forcing_reference_concentration'], 273.021047, specie='N2O')

In [ ]:
f.species_configs

In [ ]:
# set initial conditions
initialise(f.concentration, f.species_configs['baseline_concentration'])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.airborne_emissions, 0)
initialise(f.cumulative_emissions, 0)

In [ ]:
f.run()

In [ ]:
f.forcing

In [ ]:
pl.plot(f.temperature[100:,0,:,0] - f.temperature[100,0,:,0]);

In [ ]:
pl.plot(f.temperature[100:,1,:,0] - f.temperature[100,1,:,0]);

In [ ]:
pl.plot(f.temperature[100:,2,:,0] - f.temperature[100,2,:,0]);

In [ ]:
# ZEC50
pl.hist(f.temperature[150,0,:,0] - f.temperature[100,0,:,0], alpha=0.4, bins=np.arange(-0.8, 1.1, 0.02))
pl.hist(f.temperature[150,1,:,0] - f.temperature[100,1,:,0], alpha=0.4, bins=np.arange(-0.8, 1.1, 0.02))
pl.hist(f.temperature[150,2,:,0] - f.temperature[100,2,:,0], alpha=0.4, bins=np.arange(-0.8, 1.1, 0.02))
print(np.percentile(f.temperature[150,0,:,0] - f.temperature[100,0,:,0], (5,50,95)))
print(np.percentile(f.temperature[150,1,:,0] - f.temperature[100,1,:,0], (5,50,95)))
print(np.percentile(f.temperature[150,2,:,0] - f.temperature[100,2,:,0], (5,50,95)))

In [ ]:
# ZEC100
pl.hist(f.temperature[200,0,:,0] - f.temperature[100,0,:,0], alpha=0.4, bins=np.arange(-0.8, 1.1, 0.02))
pl.hist(f.temperature[200,1,:,0] - f.temperature[100,1,:,0], alpha=0.4, bins=np.arange(-0.8, 1.1, 0.02))
pl.hist(f.temperature[200,2,:,0] - f.temperature[100,2,:,0], alpha=0.4, bins=np.arange(-0.8, 1.1, 0.02))
print(np.percentile(f.temperature[200,0,:,0] - f.temperature[100,0,:,0], (5,50,95)))
print(np.percentile(f.temperature[200,1,:,0] - f.temperature[100,1,:,0], (5,50,95)))
print(np.percentile(f.temperature[200,2,:,0] - f.temperature[100,2,:,0], (5,50,95)))

In [ ]:
pl.plot(f.concentration[:,0,:,0]);

In [ ]:
f.temperature[150,0,12,0] - f.temperature[100,0,12,0]

In [ ]:
f.temperature[150,0,:,0] - f.temperature[100,0,:,0]

In [ ]:
df = pd.DataFrame(
    np.array([
        f.temperature[148,0,:,0] - f.temperature[100,0,:,0], 
        f.temperature[150,0,:,0] - f.temperature[100,0,:,0], 
        f.temperature[200,0,:,0] - f.temperature[100,0,:,0],
        f.temperature[150,1,:,0] - f.temperature[100,1,:,0], 
        f.temperature[200,1,:,0] - f.temperature[100,1,:,0],
    ]).T,
    index=fair_params_df.index, 
    columns=['zec48_750', 'zec50_750', 'zec100_750', 'zec50_1000', 'zec100_1000']
)

In [ ]:
os.makedirs('../results/', exist_ok=True)
df.to_csv('../results/zec_esm-bell_cal-1.4.0.csv')